In [1]:
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
from itertools import product
import matplotlib.pyplot as plt

In [2]:
def func_rastrigin(X):
    A = 10
    n = len(X)
    return A*n + sum([(x**2 - A * np.cos(2 * math.pi * x)) for x in X])

In [3]:
def func_griewank(xs):
    sum = 0
    for x in xs:
        sum += x * x
    product = 1
    for i in range(len(xs)):
        product *= math.cos(xs[i] / math.sqrt(i + 1))
    return 1 + sum / 4000 - product

In [4]:
def func_rosenbrock(X):
    sum = 0
    for i in range(len(X) - 1):
        sum += 100 * ((X[i+1] - X[i]**2)**2) + (1 - X[i])**2
    return sum 

In [5]:
def func_six_hump_camel_back(x):
    x1 = x[0]
    x2 = x[1]
    f = (4 - 2.1*(x1*x1) + (x1*x1*x1*x1)/3.0)*(x1*x1) + x1*x2 + (-4 + 4*(x2*x2))*(x2*x2)
    return f

In [6]:
class GeneticAlgorithm:
    def __init__(self, a=None, b=None, n=None, pop_size = 50, cross_rate=0.25,
                 mutation_rate=0.01, func=None, precision = 2):
        self.a = a
        self.b = b
        C = 750
        self.func_to_maximize = func
        self.func_to_optimize = lambda X : func(self.binaryToDouble(X))
        self.precision = precision
        self.N = int(np.log2((self.b - self.a) *  10 ** self.precision))
        self.n = n
        self.m = self.N * self.n
        self.pop_size = pop_size
        self.cross_rate = cross_rate
        self.mutation_rate = mutation_rate
        self.number_of_evaluations = 0
    
    
    def func(self, X):
        self.number_of_evaluations += 1
        return  - self.func_to_optimize(X)
    
        
    def binaryToDouble(self, byte_array):
        byte_array = str(byte_array)
        X = [ byte_array[x:x+self.N] for x in range(0, len(byte_array), self.N)]
        doubles = np.array([int(xi, 2) / (2**self.N -1) * (self.b - self.a) + self.a for xi in X]) 
        return doubles
    
    def generatePopulation(self):
        population = []
        for i in range(self.pop_size):
             population.append(self.generateRandomBinary())
        return population
    
    def generateRandomBinary(self):
        bytes = '';
        for i in range(self.m):
            bytes += str(random.randint(a=0, b=1))
        return bytes;    
    
    def changeBit(self, bite_array, position = None):
        bit_length = len(bite_array)
        index = position
        if(position is None):
            index = np.random.randint(bit_length)
        s = list(bite_array)
        if s[index] == "0":
            s[index] = "1"
        else:
            s[index] = "0"
        return "".join(s)
    
    
    def run(self, number_of_generations = 1, best_improvement = False, selection_type='fitness'):
        self.number_of_generations = number_of_generations
        self.best_improvement = best_improvement
        self.selection_type = selection_type
        return self.run_genetic_algorithm()
        
        
    def evaluatePopulation(self):
        return self._evaluatePopulation(self.population)
    
    def _evaluatePopulation(self, population):
        evaluatePopulation = []
        for chromosome in population:
            value = self.func(chromosome)
            evaluatePopulation.append(value)
        return evaluatePopulation
            
    
    
    def wheel_of_fortune_based_on_rank(self):
        minimum = np.min(self.population_evaluation)
        population_evaluation = self.population_evaluation[:]
        
        indexes = np.argsort(self.population_evaluation[:])
#         print(indexes)
        
        
        population = np.asarray(self.population)[indexes]
        
        rank_population = np.arange(1, self.pop_size)
        
          
        total_fitness = np.sum(rank_population)
        fitness_per_chromosome = rank_population / total_fitness
        
        
        q = []
        q.append(0.0)
        cumulative_probability = 0
        for fitness in fitness_per_chromosome:
            cumulative_probability += fitness
            q.append(cumulative_probability)
        
        selection = []
        for j in range(self.pop_size):
            fortune =  random.uniform(0, 1)
            for i in range(self.pop_size):
                if q[i] <= fortune and fortune <= q[i+1]:
                    selection.append(population[i][:])
        return selection
    
    
    
    def wheel_of_fortune(self):
        minimum = np.min(self.population_evaluation)
        population_evaluation = self.population_evaluation[:]
        if(minimum < 0):
            C = -1 * minimum * 1.001
            for i in range(len(population_evaluation)):
                population_evaluation[i] = population_evaluation[i] + C
          
        total_fitness = np.sum(population_evaluation)
        fitness_per_chromosome = np.asarray(population_evaluation) / total_fitness
        q = []
        q.append(0.0)
        cumulative_probability = 0
        for fitness in fitness_per_chromosome:
            cumulative_probability += fitness
            q.append(cumulative_probability)
        selection = []
        for j in range(self.pop_size):
            fortune =  random.uniform(0, 1)
            for i in range(self.pop_size):
                if q[i] <= fortune and fortune <= q[i+1]:
                    selection.append(self.population[i][:])
        return selection
    
    def cross_over(self):
        indexes_for_cross_over = []
        for i in range(len(self.population)):
            if(random.uniform(0, 1) < self.cross_rate):
                indexes_for_cross_over.append(i)
        
        #np.random.shuffle(indexes_for_cross_over)
        
        if(len(indexes_for_cross_over) % 2 == 1):
            indexes_for_cross_over.pop()
        
        for i in range(0, len(indexes_for_cross_over), 2):
            cross_point = np.random.randint(1, self.m)
            idx1 = indexes_for_cross_over[i]
            idx2 = indexes_for_cross_over[i + 1]
            
            parent1 = self.population[idx1]
            parent2 = self.population[idx2]
            
            x1 = parent1[:cross_point]
            x2 = parent1[cross_point:]
            y1 = parent2[:cross_point]
            y2 = parent2[cross_point:]
            
            self.population[idx1] = x1 + y2
            self.population[idx2] = x2 + y1
    
    def mutation(self):
        for i in range(len(self.population)):
            for mutation_point in range(self.m):
                if(random.uniform(0, 1) < self.mutation_rate):
                    self.population[i] = self.changeBit(self.population[i], mutation_point)
    
    
    def printBestCurrent(self):
        idx = np.argmax(self.population_evaluation)
        #print("Best current value = " + str(self.func_to_optimize(self.population[idx])))
    
    
    
    def run_genetic_algorithm(self):
        self.population = self.generatePopulation()
        self.population_evaluation = self.evaluatePopulation()
        max = - math.inf
        values = []
        value = None
        x = None
        idx = None
        print("running")
        for i in range(self.number_of_generations):
            if self.selection_type == 'rank':
                self.population = self.wheel_of_fortune()
            else:
                self.population = self.wheel_of_fortune_based_on_rank()
            
            self.cross_over()
            self.mutation()
            self.population_evaluation = self.evaluatePopulation()
            idx = np.argmax(self.population_evaluation)
            if(max < self.population_evaluation[idx]):
                max = self.population_evaluation[idx]
                value = self.func_to_optimize(self.population[idx])
                x = self.population[idx]
            values.append(self.func_to_optimize(self.population[idx]))
            self.printBestCurrent()
        #print(len(self.population))
        #print("Best value = " + str(value))
        #print("Last value = " + str(self.func_to_optimize(self.population[idx])))
        return values, self.number_of_evaluations, values[-1]

In [7]:
rastragin_values = []
for i in range(30):
    model = GeneticAlgorithm(a = -5.12, b = 5.12, pop_size=100, cross_rate=0.0025,
                         mutation_rate=0.001, func = func_rastrigin, n = 30)
    values, number_of_evaluations, last_value = model.run(number_of_generations=1000, selection_type='fitness')
    rastragin_values.append(last_value)
# plt.plot(values)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [13]:
rastragin_values

[98.20356438477725,
 77.99144280600348,
 89.4299070603825,
 80.11844553803937,
 59.57185267712032,
 56.86012298917731,
 77.06959327321067,
 63.09937929971608,
 91.98862852668734,
 74.17522925609103,
 85.06497178354937,
 71.83273437206736,
 79.71240197369076,
 69.71930408425791,
 101.76857690866416,
 98.30644218234085,
 81.32934298846592,
 77.88507804047205,
 98.03022915782358,
 92.008125338403,
 65.07304553165625,
 89.88344689876462,
 78.82405584357144,
 79.91944673245516,
 85.9048589273574,
 75.12291627533003,
 71.18708993516526,
 75.85353018222744,
 83.64333395144948,
 92.89777834061982]

In [8]:
last_value, number_of_evaluations

(92.89777834061982, 100100)

In [9]:
griewank_values = []
for i in range(30):
    model = GeneticAlgorithm(a = -600, b = 600, pop_size=100, cross_rate=0.0025,
                             mutation_rate=0.001, func = func_griewank, n = 30)
    values, number_of_evaluations,last_value = model.run(number_of_generations=1000, selection_type='fitness')
    griewank_values.append(last_value)
# plt.plot(values)


running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [14]:
griewank_values

[6.726794500012239,
 8.059192335079814,
 7.15701605453291,
 2.474799273926457,
 1.7792102502423517,
 2.8050530291301072,
 6.868967451413973,
 4.310673203307537,
 8.057359328408497,
 8.07041587777593,
 6.681642423946927,
 8.091901476162272,
 7.051764401841984,
 1.5935721696805514,
 1.5855374674811753,
 7.037600459429024,
 7.04860540574759,
 2.61130097032205,
 2.578150952253567,
 2.487704711086516,
 1.3557979427516695,
 2.5944247790061454,
 12.388600915130976,
 1.548651297179502,
 1.479964467862695,
 7.072175864455404,
 7.071135892791644,
 9.818087440914466,
 2.566091187112145,
 6.773578448734772]

In [10]:
values

[643.8166809784125,
 664.8738239974383,
 671.6852013271706,
 675.7996102026476,
 631.5996834371866,
 600.8650008452386,
 614.1489712811403,
 611.8303132188178,
 580.0603250255797,
 598.5931648716615,
 598.5931648716615,
 608.0717963878442,
 567.6297629824031,
 606.3858595598407,
 556.1896205822002,
 568.1302979053745,
 555.47128380809,
 548.0297735648804,
 555.5444270299091,
 557.0520696362474,
 505.7148909106305,
 521.8296793808377,
 493.46923107993564,
 525.6641890331895,
 467.5871273403524,
 474.7015744581907,
 459.7109127961471,
 474.7015744581907,
 472.74888816151366,
 438.57135497884747,
 431.68054721545377,
 430.8607643206712,
 443.0443462520624,
 442.31606086094183,
 420.5050146528077,
 392.47523598517915,
 409.03694826503727,
 411.47280584020655,
 376.6685681325322,
 383.9700831169523,
 383.9700831169523,
 383.9742280956255,
 394.77206824619105,
 394.6846476933943,
 360.8194587319544,
 343.2789538719239,
 336.3790975747694,
 357.58519531551343,
 383.0887086188922,
 370.4686182

In [11]:
rosenbrock_values = []
for i in range(30):
    model = GeneticAlgorithm(a = -2.04, b = 2.04, pop_size=100, cross_rate=0.0025,
                         mutation_rate=0.001, func = func_rosenbrock, n = 30)
    values, number_of_evaluations, last_value = model.run(number_of_generations=1000, selection_type='fitness')
    rosenbrock_values.append(last_value)
# plt.plot(values)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [15]:
rosenbrock_values

[194.2858907648001,
 93.88280504319997,
 183.32254535679996,
 159.6488474624,
 102.9594038272,
 230.13597511679998,
 186.46632857599988,
 149.4977032192,
 116.34200166399995,
 363.96565299199995,
 234.50145423359996,
 34.202893107200005,
 241.2980867072,
 263.4762592256,
 217.94819072000004,
 433.33517680639994,
 151.20251904,
 201.05540034560002,
 210.1665697792,
 36.403612876800004,
 314.8236951552,
 183.93396019199997,
 299.1795470335999,
 83.65294592000001,
 256.9286070272,
 342.7363176447999,
 225.4422274048,
 239.7684535295999,
 292.3212193792001,
 221.13633157119992]

In [12]:
six_hump_camel_back_values = []
for i in range(30):
    model = GeneticAlgorithm(a = -2, b = 2, pop_size=100, cross_rate=0.0025,
                             mutation_rate=0.001, func = func_six_hump_camel_back, n = 2)
    values, number_of_evaluations, last_value = model.run(number_of_generations=1000, selection_type='fitness')
    six_hump_camel_back_values.append(last_value)
# plt.plot(values)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


In [16]:
six_hump_camel_back_values

[-0.9260475845498222,
 -0.9896702444405533,
 -0.7336007861577155,
 -0.9623626297870567,
 -1.011146133843033,
 -0.9896702444405533,
 -0.9807123004589706,
 -0.993629773211391,
 -1.0167945961900122,
 -0.75316790646614,
 -0.7340462297217001,
 -0.993629773211391,
 -1.0286520466075206,
 -0.9689538092599873,
 -1.0289073396697246,
 -1.0243603718669945,
 -0.9896702444405533,
 -1.0113795423433776,
 -0.8358294661333733,
 -1.006155424519532,
 -1.0226989850138912,
 -0.9260475845498222,
 -0.973648243093002,
 -1.0206927672367672,
 -0.9689538092599873,
 -0.9938023552233667,
 -0.9272232937340128,
 -1.0289073396697246,
 -0.9936297732113909,
 -1.0226989850138912]